In [1]:
import praw
from textblob import TextBlob
import pandas as pd
import yfinance as yf
from pymongo import MongoClient
import time
import requests

In [2]:
# --- Reddit API setup ---
reddit = praw.Reddit(
    client_id="K2jdMrTGTjoqJs-SyJhtcA",
    client_secret="9xP07lyTS3RPiLCS2QrnYgsOq6CJ4A",
    user_agent="MyRedditScraper/0.1"
)


In [11]:
client = MongoClient("mongodb://localhost:27017/")
db = client["RedditCSR"]
collection = db["sp500_csr_esg_posts"]

In [ ]:
# Get S&P 500 company list
# URL for S&P 500 list
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Add headers to avoid 403 error
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/126.0.0.0 Safari/537.36"
}

# Request the page with a fake browser header
response = requests.get(url, headers=headers)
response.raise_for_status()  # will raise error if the request failed

# Parse all tables on the page
tables = pd.read_html(response.text)

# The first table is the one we want
sp500_companies = tables[0]

# --- Connect to MongoDB ---
client = MongoClient("mongodb://localhost:27017/")
db = client["FinanceDB"]
collection = db["SP500_Companies"]

# --- Insert into MongoDB ---
# Clear existing data (optional)
collection.delete_many({})

# Convert DataFrame to dictionary and insert
data = sp500_companies.to_dict("records")
collection.insert_many(data)

print(f"✅ Successfully stored {len(data)} S&P 500 companies in MongoDB!")

C:\Users\HomePC\AppData\Local\Temp\ipykernel_16032\957618259.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


✅ Successfully stored 503 S&P 500 companies in MongoDB!


In [18]:
# 4️⃣  Define search parameters
esg_keywords = ["CSR", "ESG", "sustainability", "corporate social responsibility"]
subreddits = ["investing", "stocks", "finance", "sustainability", "business"]

In [ ]:
def search_reddit(company_name):
    """Search Reddit posts mentioning company + ESG-related keywords"""  
    results = []
    for subreddit in subreddits:
        for keyword in esg_keywords:
            query = f'"{company_name}" {keyword}'
            print(f"🔍 Searching: {query} in r/{subreddit}")
            try:
                for submission in reddit.subreddit(subreddit).search(query, limit=5, sort="relevance"):
                    sentiment = TextBlob(submission.title + " " + submission.selftext).sentiment.polarity
                    results.append({
                        "company": company_name,
                        "subreddit": subreddit,
                        "title": submission.title,
                        "text": submission.selftext,
                        "score": submission.score,
                        "num_comments": submission.num_comments,
                        "url": submission.url,
                        "created_utc": submission.created_utc,
                        "sentiment": sentiment
                    })
            except Exception as e:
                print(f"⚠️ Error while searching {company_name}: {e}")
            time.sleep(2)  # avoid rate limits
    return results


# 6️⃣  Run the search and store results
# -----------------------
for _, row in companies.iterrows():
    company_name = row["Security"]
    posts = search_reddit(company_name)
    if posts:
        collection.insert_many(posts)
        print(f"✅ Stored {len(posts)} posts for {company_name}")
    else:
        print(f"❌ No posts found for {company_name}")

print("🎯 Done! All results stored in MongoDB collection: Reddit_ESG_Posts")
  

🔍 Searching: "3M" CSR in r/investing
🔍 Searching: "3M" ESG in r/investing
🔍 Searching: "3M" sustainability in r/investing
🔍 Searching: "3M" corporate social responsibility in r/investing
🔍 Searching: "3M" CSR in r/stocks
🔍 Searching: "3M" ESG in r/stocks
🔍 Searching: "3M" sustainability in r/stocks
🔍 Searching: "3M" corporate social responsibility in r/stocks
🔍 Searching: "3M" CSR in r/finance
🔍 Searching: "3M" ESG in r/finance
🔍 Searching: "3M" sustainability in r/finance
🔍 Searching: "3M" corporate social responsibility in r/finance
🔍 Searching: "3M" CSR in r/sustainability
🔍 Searching: "3M" ESG in r/sustainability
🔍 Searching: "3M" sustainability in r/sustainability
🔍 Searching: "3M" corporate social responsibility in r/sustainability
🔍 Searching: "3M" CSR in r/business
🔍 Searching: "3M" ESG in r/business
🔍 Searching: "3M" sustainability in r/business
🔍 Searching: "3M" corporate social responsibility in r/business
✅ Stored 21 posts for 3M
🔍 Searching: "A. O. Smith" CSR in r/investin